In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time
import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)

await asyncio.gather(attcs.start_task, latiss.start_task)

In [ ]:
await salobj.set_summary_state(attcs.atmcs, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(attcs.athexapod, salobj.State.ENABLED)

In [ ]:
await attcs.enable({"athexapod": "ncsa_202002.yaml"})

In [ ]:
await salobj.set_summary_state(latiss.atspectrograph, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(latiss.atarchiver, salobj.State.ENABLED, settingsToApply="normal")

In [ ]:
await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED, settingsToApply="Normal")
#await latiss.atcamera.cmd_enterControl.start()

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'
latiss.atcamera.cmd_takeImages.set(numImages=1,
                                   expTime=0.,
                                   shutter=False,
                                   imageType="BIAS",
                                   groupId=group_id,
                                   science=True,
                                   guide=False,
                                   wfs=False)
await latiss.atcamera.cmd_takeImages.start(timeout=60)